# To do:

 - Figure out how to signal injuries
 - Add team shooting percentages for AST stats
 - Add def team shooting percentages for REB stats
 - Add TOV stats for STL

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib
import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")
now = str(datetime.now().date())
print(f"Today's date:", now)
tgt_stat = "PTS"
print('Target Stat:', tgt_stat)

Today's date: 2025-12-29
Target Stat: PTS


In [2]:
%run ./common_utils.ipynb

# ML Functions

In [3]:
def feature_importance(model):
    importance = model.get_score(importance_type='gain')

    # Convert to table
    df_importance = (
        pd.DataFrame({
            'feature': list(importance.keys()),
            'importance': list(importance.values())
        })
        .sort_values(by='importance', ascending=False)
        .reset_index(drop=True)
    )

    df_importance['pct'] = df_importance.importance.cumsum() / df_importance.importance.sum()
    display(df_importance)

    xgb.plot_importance(model)
    plt.show()

In [4]:
def create_baseline_model(df, pred_col, DFS):
    
    train_df, val_df, test_df = DFS

    if pred_col == 'MP':
        print('Minutes Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            'starter', 'bench', 'reserve'
        ]
    else:
        print(f'{pred_col} Stats Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            f'{pred_col}_last_3_avg', f'{pred_col}_last_5_avg', f'{pred_col}_last_10_avg',
            f'Def_{pred_col}', f'Def_L5_{pred_col}'
        ]
    
    print('Train:', len(train_df), '/ Validation:', len(val_df), '/ Test:', len(test_df))
    
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert to DMatrix (XGBoost internal format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42
    }

    # Train using native XGBoost API with early stopping
    evals = [(dtrain, "train"), (dval, "val")]
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict on test set
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R²:", r2)
    
    return bst

In [5]:
def hyperparam_tuning(DFS, pred_col, n_iter=20, early_stopping_rounds=50):
    """
    Hyperparameter tuning using native XGBoost API and DMatrix,
    with early stopping support (compatible with XGBoost 3.1.2)
    """

    train_df, val_df, test_df = DFS
    feature_cols = [col for col in train_df.columns if col not in ['Date', pred_col]]
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert datasets to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    # Hyperparameter search space
    param_dist = {
        "n_estimators": randint(300, 1500),
        "learning_rate": uniform(0.01, 0.05),
        "max_depth": randint(3, 6),
        "min_child_weight": randint(1, 8),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.7, 0.3),
        "gamma": uniform(0, 2),
        "reg_lambda": uniform(0, 5),
        "reg_alpha": uniform(0, 2)
    }

    # Sample n_iter random parameter combinations
    param_list = []
    for _ in range(n_iter):
        sample = {k: (v.rvs() if hasattr(v, "rvs") else v) for k, v in param_dist.items()}
        sample['n_estimators'] = int(sample['n_estimators'])
        sample['max_depth'] = int(sample['max_depth'])
        sample['min_child_weight'] = int(sample['min_child_weight'])
        param_list.append(sample)

    best_mae = float('inf')
    best_params = None
    best_bst = None

    # Manual hyperparameter search
    for i, params in enumerate(param_list):
        print(f"\nTrial {i+1}/{n_iter}: {params}")
        num_boost_round = params.pop('n_estimators')
        params.update({
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "device": "cuda",
            "seed": 42
        })
        evals = [(dtrain, 'train'), (dval, 'val')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )
        # Predict on validation set to compute MAE
        val_preds = bst.predict(dval, iteration_range=(0, bst.best_iteration))
        mae = mean_absolute_error(y_val, val_preds)
        print(f"Validation MAE: {mae:.4f}")
        if mae < best_mae:
            best_mae = mae
            best_params = params.copy()
            best_bst = bst

    print("\nBest validation MAE:", best_mae)
    print("Best parameters:", best_params)

    # Predict on test set using best model
    preds = best_bst.predict(dtest, iteration_range=(0, best_bst.best_iteration))
    test_df[pred_col] = y_test
    test_df[f'{pred_col}_preds'] = preds
    test_df['Team'] = team_encoder.inverse_transform(test_df["Team"])
    test_df['Opp'] = team_encoder.inverse_transform(test_df["Opp"])
    test_df['Player'] = player_encoder.inverse_transform(test_df["Player"])
    test_df['Pos'] = position_encoder.inverse_transform(test_df["Pos"])
    analyze_df = test_df[['Date', 'Team', 'Player', 'Pos', 'Opp', pred_col, f'{pred_col}_preds']]
    print("\nTest Metrics:")
    print("RMSE:", np.sqrt(mean_squared_error(y_test, preds)))
    print("MAE:", mean_absolute_error(y_test, preds))
    print("R²:", r2_score(y_test, preds))

    return best_bst, preds, y_test, analyze_df

### Create Base df

In [6]:
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for i in [2022, 2023, 2024, 2025]:
    df_actuals = pd.read_csv(f"../tables/{i}/parlay_actuals.csv")
    df_actuals['Season'] = i
    df = pd.concat([df, df_actuals])

    df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
    df_schd['Season'] = i
    df2 = pd.concat([df2, df_schd])
    
    df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
    df_gms['Season'] = i
    df3 = pd.concat([df3, df_gms])
    
    df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
    df_inj['Season'] = i
    df4 = pd.concat([df4, df_inj])

df['Date'] = pd.to_datetime(df.Date)
df2['Date'] = pd.to_datetime(df2.Date)
df3['Date'] = pd.to_datetime(df3.Date)
df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
df4['Date'] = pd.to_datetime(df4.Date)

df['Tms'] = df['game_id'].apply(lambda x: x.split("_")[1:3])
df['WrngTm'] = df.apply(lambda row: 0 if row['Team'] in row['Tms'] else 1, axis=1)
df['WrngOpp'] = df.apply(lambda row: 0 if row['Opp'] in row['Tms'] else 1, axis=1)
df = df[(df.WrngTm == 0) & (df.WrngOpp == 0)].drop(['WrngTm', 'WrngOpp', 'Tms'], axis=1)

df3_temp = df3[['game_id', 'Date', 'Team', 'Player', 'Active', 'FG', 'FGA', 'FG%', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TOV', 'PF', '+/-']]\
        .rename(columns={"3PA": "TPA", "3P%": "TP%"})
df = df.merge(df3_temp, on=['game_id', 'Date', 'Team', 'Player'])

df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch['Team_type'] = 'Away'
df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch2['Team_type'] = 'Home'
df_mtch = pd.concat([df_mtch, df_mtch2])
df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch = df_mtch.sort_values(["Team", "Date"])
df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
df = df.drop(['Spread', 'Total'], axis=1).merge(df_mtch, on=['Season', 'Date', 'Team'])

df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
df['Status'] = np.where((df.Active == 1), 'Available', df.Status)
df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

team_encoder = LabelEncoder()
player_encoder = LabelEncoder()
team_type_encoder = LabelEncoder()
position_encoder = LabelEncoder()

# Encode string cols
team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
df["Team"] = team_encoder.transform(df["Team"])
df["Opp"] = team_encoder.transform(df["Opp"])
df["Player_name"] = df.Player
df["Player"] = player_encoder.fit_transform(df["Player"])
df["Pos"] = position_encoder.fit_transform(df["Pos"])
df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
df = df[(df.Active == 1) & (df.MP > 0)].sort_values(['Season', 'Date', 'Team', 'Player']).reset_index(drop=True)
print('base df created', datetime.now())

base df created 2025-12-29 15:06:04.516577


# Minutes Projection Model

In [7]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [8]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['reserve_td', 'bench_td', 'starter_td', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

In [9]:
df_mins = df.copy()
df_mins = setup_df_mins(con, df_mins)
display(df_mins)

n = len(df_mins)
train_end = int(0.8 * n)
val_end   = int(0.9 * n)
mins_train_df = df_mins.iloc[:train_end]
mins_val_df   = df_mins.iloc[train_end:val_end]
mins_test_df  = df_mins.iloc[val_end:]
mins_DFS = (mins_train_df, mins_val_df, mins_test_df)

mins_model = create_baseline_model(df_mins, "MP", mins_DFS)
mins_model, mins_preds, y_test_mins, analyze_df_mins = hyperparam_tuning(mins_DFS, "MP", n_iter=1)
# feature_importance(mins_model)

,Season,Date,Team,Team_type,Opp,Player,Pos,B2B,MP,Spread,Total,team_game_num,cup_gm,pstszn_gm,is_OT,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,MP_change_L1,MP_change_L3,MP_pct_change,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,TOV_change_L1,TOV_change_L3,TOV_pct_change,PF_lst_gm,PF_last_3_avg,PF_last_3_std,PF_last_5_avg,PF_last_5_std,PF_last_10_avg,PF_last_10_std,PF_change_L1,PF_change_L3,PF_pct_change,+/-_lst_gm,+/-_last_3_avg,+/-_last_3_std,+/-_last_5_avg,+/-_last_5_std,+/-_last_10_avg,+/-_last_10_std,+/-_change_L1,+/-_change_L3,+/-_pct_change,MP_spike,MP_drop,MP_trend,games_last_7_days,prev_team_mins_pct,reserve,bench,starter,missed_games,blowout,DaysLstGm,travel_km,travel_hours,is_long_trip,same_arena
0,2022,2022-10-21,0,1,21,5,2,0,14.37,10.0,206.0,2,0,0,0,NaN,20.166667,NaN,24.286,NaN,26.067000,NaN,NaN,-5.900333,NaN,NaN,0.666667,NaN,1.0,NaN,1.400000,NaN,NaN,-0.733333,NaN,NaN,3.333333,NaN,2.4,NaN,2.000000,NaN,NaN,1.333333,NaN,NaN,18.333333,NaN,20.2,NaN,14.100000,NaN,NaN,4.233333,NaN,0,0,-5.900333,3,NaN,0,1,0,0,0,0,0.0,0.0,0,0
1,2022,2022-10-21,0,1,21,120,0,0,31.62,10.0,206.0,2,0,0,0,NaN,4.266667,NaN,7.614,NaN,19.136000,NaN,NaN,-14.869333,NaN,NaN,0.000000,NaN,0.0,NaN,0.300000,NaN,NaN,-0.300000,NaN,NaN,0.000000,NaN,0.0,NaN,0.400000,NaN,NaN,-0.400000,NaN,NaN,3.333333,NaN,3.2,NaN,-1.500000,NaN,NaN,4.833333,NaN,0,0,-14.869333,1,NaN,0,0,1,0,0,0,0.0,0.0,0,0
2,2022,2022-10-21,0,1,21,171,3,0,32.53,10.0,206.0,2,0,0,0,NaN,26.603333,NaN,29.612,NaN,30.320000,NaN,NaN,-3.716667,NaN,NaN,1.333333,NaN,2.0,NaN,2.900000,NaN,NaN,-1.566667,NaN,NaN,2.333333,NaN,2.8,NaN,2.300000,NaN,NaN,0.033333,NaN,NaN,8.333333,NaN,6.4,NaN,4.400000,NaN,NaN,3.933333,NaN,0,0,-3.716667,4,NaN,0,0,1,0,0,0,0.0,0.0,0,0
3,2022,2022-10-21,0,1,21,178,4,0,39.62,10.0,206.0,2,0,0,0,NaN,32.200000,NaN,30.734,NaN,30.403000,NaN,NaN,1.797000,NaN,NaN,1.333333,NaN,0.8,NaN,1.000000,NaN,NaN,0.333333,NaN,NaN,2.333333,NaN,2.8,NaN,2.000000,NaN,NaN,0.333333,NaN,NaN,-0.666667,NaN,-0.4,NaN,1.300000,NaN,NaN,-1.966667,NaN,0,0,1.797000,2,NaN,0,0,1,0,0,0,0.0,0.0,0,0
4,2022,2022-10-21,0,1,21,321,3,0,11.77,10.0,206.0,2,0,0,0,NaN,23.016667,NaN,22.238,NaN,17.904286,NaN,NaN,5.112381,NaN,NaN,2.333333,NaN,1.8,NaN,1.428571,NaN,NaN,0.904762,NaN,NaN,1.333333,NaN,1.4,NaN,1.428571,NaN,NaN,-0.095238,NaN,NaN,-6.000000,NaN,-3.2,NaN,-1.142857,NaN,NaN,-4.857143,NaN,0,0,5.112381,2,NaN,0,1,0,0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80261,2025,2025-12-28,25,0,13,523,4,1,19.40,-24.0,226.0,32,0,0,0,26.75,22.016667,11.298147,22.644,10.916167,24.230000,8.116632,4.106,-2.213333,0.104003,0.0,2.333333,0.577350,1.6,0.547723,1.400000,1.840894,-1.6,0.933333,-0.999999,3.0,1.666667,1.732051,2.0,1.341641,1.700000,1.173788,1.0,-0.033333,0.764705,-4.0,-11.333333,4.163332,-13.2,14.737707,-6.300000,15.049548,9.2,-5.033333,-0.365079,0,0,-2.213333,3,0.111458,0,0,1,5,0,14,0.0,0.0,0,0
80262,2025,2025-12-28,25,0,13,550,0,1,34.63,-24.0,226.0,32,0,0,0,37.10,34.823333,13.541879,33.500,11.769737,29.238000,8.104324,3.600,5.585333,0.268897,2.0,3.000000,1.154701,2.4,0.836660,1.800000,1.766981,-0.4,1.200000,0.111111,6.0,4.000000,1.732051,3.8,2.121320,3.300000,1.619328,2.2,0.700000,0.818182,3.0,-7.000000,3.785939,-13.2,4.335897,-6.300000,15.107026,16.2,-0.700000,-1.476191,0,0,5.585333,3,0.154583,0,1,0,2,0,7,0.0,0.0,0,0
80263,2025,2025-12-28,25,0,13,596,4,1,28.92,-24.0,226.0,32,0,0,0,26.90,20.056667,5.932748,20.540,10.067951,17.092000,8.113332,6.360,2.964667,0.573836,1.0,1.000000,1.000000,1.8,0.836660,1.300000,1.776388,-0.8,-0.300000,-0.230769,2.0,2.000000,2.081666,1.8,2.167948,2.000000,1.577621,0.2,0.000000,0.000000,4.0,-3.666667,4.358899,-8.4,3.346640,-6.400000,15.512003,12.4,2

Minutes Model
Train: 64212 / Validation: 8027 / Test: 8027
RMSE: 5.983898955178516
MAE: 4.58646926348572
R²: 0.6475748149500435

Trial 1/1: {'n_estimators': 591, 'learning_rate': np.float64(0.025681216827075914), 'max_depth': 3, 'min_child_weight': 6, 'subsample': np.float64(0.8280327398052351), 'colsample_bytree': np.float64(0.7096682418142123), 'gamma': np.float64(1.3545916221944505), 'reg_lambda': np.float64(2.621927420440411), 'reg_alpha': np.float64(1.2757867781838537)}
Validation MAE: 4.9560

Best validation MAE: 4.955957942619052
Best parameters: {'learning_rate': np.float64(0.025681216827075914), 'max_depth': 3, 'min_child_weight': 6, 'subsample': np.float64(0.8280327398052351), 'colsample_bytree': np.float64(0.7096682418142123), 'gamma': np.float64(1.3545916221944505), 'reg_lambda': np.float64(2.621927420440411), 'reg_alpha': np.float64(1.2757867781838537), 'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 5.750865452194

In [10]:
rmse = np.sqrt(mean_squared_error(y_test_mins, mins_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_mins, mins_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', 'MP_last_5_avg']][df_yesterday.MP > 0]

df_yesterday['Diff'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff'] <= rmse, 1, 0)

print("\nYesterday's Results:")
print("Total Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), '/', df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff'], axis=1)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 5.750865452194383

Yesterday's Results:
Total Accuracy (InRMSE_Range): 0.8181818181818182
9 / 11


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
0,2025-12-28,LAL,Luka Doncic,PG,SAC,35.485744,32.73,35.731909,1
1,2025-12-28,DET,Cade Cunningham,PG,LAC,36.126133,32.10,35.855542,1
2,2025-12-28,DET,Jalen Duren,C,LAC,27.187101,31.43,29.982531,1
3,2025-12-28,LAC,James Harden,PG,DET,37.331711,40.00,33.186158,1
4,2025-12-28,SAC,Russell Westbrook,PG,LAL,25.505262,26.25,33.125482,1
5,2025-12-28,LAL,LeBron James,SF,SAC,35.170448,28.63,32.810431,0
6,2025-12-28,SAC,Precious Achiuwa,C,LAL,10.727637,19.58,18.176271,0
7,2025-12-28,LAC,Kawhi Leonard,SF,DET,37.063404,38.63,30.802118,1
8,2025-12-28,DET,Ausar Thompson,SF,LAC,23.884089,23.68,26.455087,1
9,2025-12-28,LAL,Deandre Ayton,C,SAC,30.635414,30.17,27.253936,1


# Main Model

In [11]:
def setup_df_main(df):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'cup_gm', 'pstszn_gm', 'is_OT']]
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        tgt_stat_cols = ['PTS']
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'cup_gm', 'pstszn_gm', 'is_OT']]
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'cup_gm', 'pstszn_gm', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA', tgt_stat] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    for col in categories + ['FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 'reserve_td', 'bench_td', 'starter_td'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

In [17]:
df_main = df.copy()
df_main = setup_df_main(df_main)
display(df_main)

n = len(df_main)
train_end = int(0.65 * n)
val_end   = int(0.85 * n)
main_train_df = df_main.iloc[:train_end]
main_val_df   = df_main.iloc[train_end:val_end]
main_test_df  = df_main.iloc[val_end:]
main_DFS = (main_train_df, main_val_df, main_test_df)

# stat_model = create_baseline_model(df_main, tgt_stat, main_DFS)
stat_model, stat_preds, y_test_stat, analyze_df_stat = hyperparam_tuning(main_DFS, tgt_stat, n_iter=1)
# feature_importance(stat_model)

,Season,Date,Team,Opp,Player,Pos,B2B,MP,PTS,Def_PTS,Def_L5_PTS,Spread,Total,cup_gm,pstszn_gm,is_OT,three_rate_L3,ft_rate_L3,eFG_L3,TS_L3,usage_proxy_L3,three_rate_L5,ft_rate_L5,eFG_L5,TS_L5,usage_proxy_L5,three_rate_L10,ft_rate_L10,eFG_L10,TS_L10,usage_proxy_L10,three_rate_weighted,ft_rate_weighted,eFG_weighted,TS_weighted,usage_proxy_weighted,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,FGA_lst_gm,FGA_last_3_avg,FGA_last_3_std,FGA_last_5_avg,FGA_last_5_std,FGA_last_10_avg,FGA_last_10_std,TPA_lst_gm,TPA_last_3_avg,TPA_last_3_std,TPA_last_5_avg,TPA_last_5_std,TPA_last_10_avg,TPA_last_10_std,FTA_lst_gm,FTA_last_3_avg,FTA_last_3_std,FTA_last_5_avg,FTA_last_5_std,FTA_last_10_avg,FTA_last_10_std,PTS_lst_gm,PTS_last_3_avg,PTS_last_3_std,PTS_last_5_avg,PTS_last_5_std,PTS_last_10_avg,PTS_last_10_std,reserve,bench,starter
0,2022,2022-10-21,0,21,5,2,0,14.37,4,18.000000,18.0,10.0,206.0,0,0,0,0.559259,0.829630,0.416667,0.519801,0.616138,0.508283,0.691717,0.475758,0.563433,0.552491,0.479785,0.587601,0.523390,0.606242,0.509510,0.536019,0.764053,0.445066,0.541535,0.586381,NaN,20.166667,NaN,24.286,NaN,26.067000,NaN,NaN,11.333333,NaN,11.4,NaN,11.2,NaN,NaN,4.000000,NaN,4.4,NaN,4.400000,NaN,NaN,3.333333,NaN,4.2,NaN,4.500000,NaN,NaN,16.000000,NaN,16.6,NaN,16.800000,NaN,0,1,0
1,2022,2022-10-21,0,21,120,0,0,31.62,8,14.000000,14.0,10.0,206.0,0,0,0,0.666667,0.000000,0.750000,0.750000,0.219526,0.700000,0.000000,0.575000,0.575000,0.294441,0.662500,0.033333,0.655247,0.658118,0.242095,0.676250,0.003333,0.688025,0.688312,0.244257,NaN,4.266667,NaN,7.614,NaN,19.136000,NaN,NaN,1.333333,NaN,2.2,NaN,4.0,NaN,NaN,1.333333,NaN,2.0,NaN,2.900000,NaN,NaN,0.000000,NaN,0.0,NaN,0.300000,NaN,NaN,1.000000,NaN,1.6,NaN,5.100000,NaN,0,0,1
2,2022,2022-10-21,0,21,171,3,0,32.53,12,8.000000,8.0,10.0,206.0,0,0,0,0.380383,0.219298,0.403010,0.445128,0.503382,0.413871,0.277220,0.460781,0.500643,0.492433,0.413396,0.304184,0.502432,0.548425,0.565033,0.393730,0.245163,0.430284,0.472112,0.506262,NaN,26.603333,NaN,29.612,NaN,30.320000,NaN,NaN,12.666667,NaN,13.2,NaN,15.2,NaN,NaN,4.666667,NaN,5.4,NaN,6.200000,NaN,NaN,2.333333,NaN,3.4,NaN,4.600000,NaN,NaN,13.666667,NaN,15.8,NaN,19.800000,NaN,0,0,1
3,2022,2022-10-21,0,21,178,4,0,39.62,20,19.000000,19.0,10.0,206.0,0,0,0,0.000000,0.134680,0.765993,0.743921,0.295629,0.000000,0.125253,0.770707,0.757689,0.298972,0.000000,0.182828,0.733081,0.735370,0.320628,0.000000,0.136667,0.764116,0.747197,0.299132,NaN,32.200000,NaN,30.734,NaN,30.403000,NaN,NaN,8.666667,NaN,8.4,NaN,8.8,NaN,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,NaN,1.333333,NaN,1.2,NaN,1.700000,NaN,NaN,13.666667,NaN,13.2,NaN,14.000000,NaN,0,0,1
4,2022,2022-10-21,0,21,321,3,0,11.77,4,8.000000,8.0,10.0,206.0,0,0,0,0.564815,0.208333,0.400463,0.423747,0.294991,0.652525,0.361364,0.394823,0.466264,0.339934,0.466089,0.543831,0.495686,0.565859,0.264007,0.581255,0.287792,0.408293,0.450714,0.305375,NaN,23.016667,NaN,22.238,NaN,17.904286,NaN,NaN,6.333333,NaN,6.8,NaN,5.0,NaN,NaN,2.666667,NaN,4.0,NaN,2.857143,NaN,NaN,1.666667,NaN,2.2,NaN,1.857143,NaN,NaN,8.333333,NaN,9.4,NaN,7.285714,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80261,2025,2025-12-28,25,13,523,4,1,19.40,11,16.826087,15.2,-24.0,226.0,0,0,0,0.500337,0.441077,0.363636,0.440692,0.483035,0.498698,0.321789,0.378332,0.434065,0.507996,0.453364,0.208560,0.432222,0.469748,0.525778,0.495148,0.382039,0.374904,0.441610,0.494798,26.75,22.016667,11.298147,22.644,10.916167,24.230000,8.116632,9.0,8.333333,6.658328,10.2,6.519202,11.9,5.513620,5.0,4.333333,4.041452,5.0,3.049590,5.100000,2.514403,7.0,4.333333,4.358899,3.0,4.438468,2.300000,3.433495,16.0,11.333333,12.220202,12.4,12.876335,13.700000,10.347302,0,0,1
80262,2025,2025-12-28,25,13,550,0,1,34


Trial 1/1: {'n_estimators': 965, 'learning_rate': np.float64(0.02440319131233018), 'max_depth': 5, 'min_child_weight': 5, 'subsample': np.float64(0.7517347345102964), 'colsample_bytree': np.float64(0.8847923523153259), 'gamma': np.float64(0.16957831694414915), 'reg_lambda': np.float64(1.1234407325631417), 'reg_alpha': np.float64(1.3987842222200624)}
Validation MAE: 3.6010

Best validation MAE: 3.601008176803589
Best parameters: {'learning_rate': np.float64(0.02440319131233018), 'max_depth': 5, 'min_child_weight': 5, 'subsample': np.float64(0.7517347345102964), 'colsample_bytree': np.float64(0.8847923523153259), 'gamma': np.float64(0.16957831694414915), 'reg_lambda': np.float64(1.1234407325631417), 'reg_alpha': np.float64(1.3987842222200624), 'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 4.9841442473281035
MAE: 3.722688674926758
R²: 0.6989330649375916


In [18]:
analyze_df_stat = main_test_df.drop([tgt_stat, f'{tgt_stat}_preds'], axis=1)\
                .merge(analyze_df_stat[['Date', 'Team', 'Player', tgt_stat, f'{tgt_stat}_preds']], on=['Date', 'Team', 'Player'])
analyze_df_stat['Diff'] = analyze_df_stat[tgt_stat] - analyze_df_stat[f'{tgt_stat}_preds']
analyze_df_stat[analyze_df_stat.MP > 38].sort_values('Diff', ascending=True).head(15)

,Season,Date,Team,Opp,Player,Pos,B2B,MP,Def_PTS,Def_L5_PTS,Spread,Total,cup_gm,pstszn_gm,is_OT,three_rate_L3,ft_rate_L3,eFG_L3,TS_L3,usage_proxy_L3,three_rate_L5,ft_rate_L5,eFG_L5,TS_L5,usage_proxy_L5,three_rate_L10,ft_rate_L10,eFG_L10,TS_L10,usage_proxy_L10,three_rate_weighted,ft_rate_weighted,eFG_weighted,TS_weighted,usage_proxy_weighted,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,FGA_lst_gm,FGA_last_3_avg,FGA_last_3_std,FGA_last_5_avg,FGA_last_5_std,FGA_last_10_avg,FGA_last_10_std,TPA_lst_gm,TPA_last_3_avg,TPA_last_3_std,TPA_last_5_avg,TPA_last_5_std,TPA_last_10_avg,TPA_last_10_std,FTA_lst_gm,FTA_last_3_avg,FTA_last_3_std,FTA_last_5_avg,FTA_last_5_std,FTA_last_10_avg,FTA_last_10_std,PTS_lst_gm,PTS_last_3_avg,PTS_last_3_std,PTS_last_5_avg,PTS_last_5_std,PTS_last_10_avg,PTS_last_10_std,reserve,bench,starter,PTS,PTS_preds,Diff
1860,2024,2025-04-13,CLE,IND,Tristan Thompson,C,0,42.70,14.221053,17.6,-8.0,244.0,0,0,2,0.000000,0.083333,0.458333,0.464715,0.471186,0.000000,0.050000,0.475000,0.478829,0.440973,0.000000,0.025000,0.463542,0.465935,0.358056,0.000000,0.067500,0.463854,0.469071,0.450809,19.98,9.793333,9.938236,8.872,9.308119,6.51200,11.137116,8.0,3.333333,4.041452,2.8,3.701351,1.900,2.838231,0.0,0.000000,0.000000,0.0,1.414214,0.000,1.475730,2.0,0.666667,1.154701,0.4,0.894427,0.200,1.523884,7.0,3.000000,3.605551,3.0,3.209361,1.90,3.887301,1,0,0,3,21.860006,-18.860006
9489,2025,2025-11-30,ATL,PHI,Onyeka Okongwu,C,0,47.35,15.190476,14.4,8.0,276.0,0,0,2,0.391534,0.108466,0.607143,0.608324,0.576733,0.433522,0.080464,0.654496,0.657608,0.513632,0.448825,0.225629,0.663438,0.678703,0.493529,0.409860,0.111781,0.626978,0.630147,0.549482,33.70,29.366667,6.204418,29.362,9.444189,29.84400,12.926151,14.0,15.333333,7.505553,14.0,7.791020,13.400,6.964194,5.0,6.000000,5.686241,6.0,4.358899,6.200,3.951090,1.0,1.666667,2.081666,1.2,4.847680,2.400,5.143496,18.0,19.333333,9.165151,18.8,11.458621,19.60,11.421714,0,0,1,10,27.346752,-17.346752
2518,2024,2025-04-23,MIA,CLE,Bam Adebayo,C,0,42.40,12.891089,12.2,-9.0,233.0,0,1,0,0.227543,0.221861,0.484984,0.493516,0.488495,0.276526,0.279784,0.550990,0.565301,0.520204,0.242249,0.250807,0.544331,0.567886,0.583760,0.243709,0.242133,0.510720,0.522488,0.507534,39.07,39.660000,10.221952,35.176,10.700403,34.20100,11.016809,22.0,17.333333,6.658328,15.8,7.362065,17.800,6.773314,5.0,4.000000,1.000000,4.2,2.408319,4.200,3.645393,6.0,4.000000,3.055050,4.4,2.449490,4.300,2.065591,24.0,18.666667,9.018500,19.4,9.273618,21.90,6.480741,0,0,1,11,27.617775,-16.617775
3725,2024,2025-05-26,MIN,OKC,Anthony Edwards,SG,0,40.93,12.891089,19.8,-2.0,254.0,0,1,0,0.477376,0.346908,0.611237,0.641648,0.600132,0.511187,0.263383,0.640075,0.657936,0.571395,0.460463,0.360547,0.561782,0.597970,0.585480,0.485828,0.323214,0.614943,0.642167,0.590046,29.70,35.446667,3.691021,36.154,7.683709,38.75400,13.967116,17.0,18.666667,7.023769,18.4,6.797058,19.700,8.962267,8.0,8.333333,4.163332,9.0,3.033150,8.900,3.984693,2.0,6.333333,1.154701,4.8,1.414214,7.000,2.110819,30.0,26.666667,12.055428,26.4,10.382678,26.90,12.547244,0,0,1,16,31.659838,-15.659838
10616,2025,2025-12-07,LAL,PHI,Austin Reaves,SG,0,39.13,14.047619,7.8,4.0,220.0,0,0,0,0.517196,0.580688,0.648810,0.715542,0.604655,0.510317,0.628413,0.735952,0.792469,0.556912,0.454022,0.613003,0.685423,0.750524,0.539471,0.508815,0.598237,0.678614,0.742118,0.583814,33.32,35.830000,7.681154,37.688,7.805798,37.19200,6.604033,18.0,17.000000,8.717798,16.2,6.870226,15.800,5.035982,8.0,8.666667,4.163332,8.2,3.633180,7.200,2.635231,17.0,11.000000,9.291573,10.8,7.224957,9.700,5.600099,36.0,32.000000,19.731531,33.4,14.142136,30.20,10.405661,0,0,1,11,26.640450,-15.640450
9862,2025,2025-12-02,NYK,BOS,Jalen Brunson,PG,0,39.22,17.250000,18.4,-6.0,240.0,0,0,0,0.271930,0.305973,0.514829,0.566411,0.722496,0.343609,0.272306,0.563784,0.612314,0.686039,0.359142,0.278386,0.559474,0.607156,0.670780,0.302155,0.293114,0.533980,0.584256,0.706388,33.73,35.593333,15.328227,35.

In [19]:
rmse = np.sqrt(mean_squared_error(y_test_stat, stat_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_stat, stat_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)
df_gms = df_gms.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df_gms['STL_BLK'] = df_gms.STL + df_gms.BLK
df_gms['PR'] = df_gms.PTS + df_gms.REB 
df_gms['PA'] = df_gms.PTS + df_gms.AST
df_gms['RA'] = df_gms.REB + df_gms.AST
df_gms['PRA'] = df_gms.PTS + df_gms.REB + df_gms.AST

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', tgt_stat, 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', f'{tgt_stat}_line', f'{tgt_stat}_proj', tgt_stat]][df_yesterday.MP > 0]

df_yesterday['Diff'] = df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[f'{tgt_stat}_line']
df_yesterday['Diff2'] = abs(df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[tgt_stat])
df_yesterday['Act_Res'] = np.where(df_yesterday[tgt_stat] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['Pred_Res'] = np.where(df_yesterday[f'{tgt_stat}_proj'] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['ParlayHit'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff2'] <= rmse, 1, 0)

print("Total Accuracy (ParlayHit):", ((df_yesterday.ParlayHit == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.ParlayHit == 1).sum(), "/", df_yesterday.shape[0])

print("\nTotal Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), "/", df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff', 'Act_Res', 'Pred_Res'], axis=1).sort_values(f'{tgt_stat}_line', ascending=False)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 4.9841442473281035
Total Accuracy (ParlayHit): 1.0
3 / 3

Total Accuracy (InRMSE_Range): 0.6666666666666666
2 / 3


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
0,2025-12-28,LAL,Luka Doncic,PG,SAC,35.549591,32.73,35.5,27.726818,34,6.273182,1,0
1,2025-12-28,SAC,Keon Ellis,SG,LAL,9.802944,22.27,9.5,2.905766,2,0.905766,1,1
2,2025-12-28,SAC,Precious Achiuwa,C,LAL,10.736481,19.58,8.5,4.277500,5,0.722500,1,1


### Today's predictions

In [20]:
df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)

df_pred = pd.read_csv("../tables/2025/parlay_stats.csv")
df_pred['Date'] = pd.to_datetime(df_pred.Date)
df_pred['Season'] = 2025
for col in df_pred.select_dtypes(include='object').columns:
    df_pred[col] = df_pred[col].astype('category')
df_pred = df_pred.drop(['Spread', 'Total'], axis=1).merge(df_mtch, on=['Season', 'Date', 'Team'])
df_pred[tgt_stat] = 0

# Predict minutes
df_act_mins = pd.read_csv("../tables/2025/parlay_actuals.csv")
df_act_mins['Date'] = pd.to_datetime(df_act_mins.Date)
df_pred = df_pred[df_pred.Player.isin(df.Player_name.unique())].merge(df_act_mins[['Date', 'Team', 'Player', 'MP', 'TPM']], on=['Date', 'Team', 'Player'], how='left')
df_temp = df.copy()
df_temp["Team"] = team_encoder.inverse_transform(df_temp["Team"])
df_temp['Player'] = player_encoder.inverse_transform(df_temp["Player"])
df_pred = df_pred.merge(df_temp[['Date', 'Team', 'Player', 'TOV', 'PF', '+/-', 'FGA', 'FG', 'TPA', 'FT', 'FTA',
                             'AST', 'REB', 'PR', 'PA', 'RA', 'STL', 'BLK', 'STL_BLK']], on=['Date', 'Team', 'Player'], how='left')

df_pred = df_pred.merge(df_lines, on=['Date', 'Team', 'Player'], how='left')
df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
df_pred_mins = setup_df_mins(con, df_pred)

df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
df_pred_mins["Team"] = team_encoder.transform(df_pred_mins["Team"])
df_pred_mins["Opp"] = team_encoder.transform(df_pred_mins["Opp"])
df_pred_mins['Team_type'] = team_type_encoder.transform(df_pred_mins['Team_type'])
df_pred_mins["Player"] = player_encoder.transform(df_pred_mins["Player"])
df_pred_mins["Pos"] = position_encoder.transform(df_pred_mins["Pos"])
DM_mins = xgb.DMatrix(df_pred_mins)
df_pred['MP'] = mins_model.predict(DM_mins)
df_pred['N_TPM'] = df_pred.FG - df_pred.TPM
df_pred['PTS'] = (df_pred.FT * 1) + (df_pred.N_TPM * 2) + (df_pred.TPM * 3)
df_pred = setup_df_main(df_pred)
feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
df_pred = df_pred[df_pred.Date == now][feature_cols]

# Predict stat
df_pred["Team"] = team_encoder.transform(df_pred["Team"])
df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
df_pred["Player"] = player_encoder.transform(df_pred["Player"])
df_pred["Pos"] = position_encoder.transform(df_pred["Pos"])
DM_stats = xgb.DMatrix(df_pred)
df_pred[f"{tgt_stat}_proj"] = stat_model.predict(DM_stats)

df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
            [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
tds_picks = tds_picks[(tds_picks.Diff >= mae) & (tds_picks.Diff2 <= 8)].sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
display(tds_picks)
tds_picks.insert(0, 'Date', pd.to_datetime(now))
partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
100,MIL,Giannis Antetokounmpo,PF,CHO,24.754818,29.029779,26.5,14.412952
77,HOU,Kevin Durant,SF,IND,30.136009,36.369845,27.5,16.283543
14,OKC,Shai Gilgeous-Alexander,PG,ATL,29.842587,32.374957,30.5,21.111624
61,PHO,Devin Booker,SG,WAS,30.792356,31.919560,27.5,18.351713
137,SAS,Victor Wembanyama,C,CLE,23.338032,25.750860,22.5,14.253018
124,NYK,Karl-Anthony Towns,C,NOP,28.076813,32.096676,24.5,16.705688
73,IND,Aaron Nesmith,SF,HOU,27.603863,30.324699,8.5,15.778637
55,MIN,Anthony Edwards,SG,CHI,32.448013,36.979648,30.5,23.563267
87,ATL,Zaccharie Risacher,SF,OKC,15.936276,21.486315,12.5,5.736259
105,HOU,Tari Eason,PF,IND,13.755330,18.248464,12.5,6.137624


../tables/2025/gmday_preds_PTS.csv saved!
